In [2]:
!conda install -c rapidsai -c nvidia -c conda-forge -c defaults rapids=0.13 python=3.6

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
| ^C
failed

CondaError: KeyboardInterrupt



In [8]:
import platform
print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(platform.processor())

x86_64
#1 SMP Debian 4.9.210-1 (2020-01-20)
Linux-4.9.0-12-amd64-x86_64-with-debian-9.12
uname_result(system='Linux', node='university-of-liverpool-ion-switching', release='4.9.0-12-amd64', version='#1 SMP Debian 4.9.210-1 (2020-01-20)', machine='x86_64', processor='')
Linux



'#1 SMP Debian 4.9.210-1 (2020-01-20)'

In [ ]:
>>> import platform
>>> platform.machine()
'x86'
>>> 
'5.1.2600'
>>> 
'Windows-XP-5.1.2600-SP2'
>>> 
('Windows', 'name', 'XP', '5.1.2600', 'x86', 'x86 Family 6 Model 15 Stepping 6, GenuineIntel')
>>> platform.system()
'Windows'
>>> platform.processor()

In [3]:
import sys
!cp ../input/rapids/rapids.0.12.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.6"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
# from scipy.stats import mode
from sklearn.metrics import f1_score, accuracy_score
from cuml.neighbors import KNeighborsClassifier, NearestNeighbors
import cuml; cuml.__version__

ImportError: No module named 'sklearn.__check_build._check_build'
___________________________________________________________________________
Contents of /opt/conda/envs/rapids/lib/python3.6/site-packages/sklearn/__check_build:
_check_build.cpython-36m-x86_64-linux-gnu.sosetup.py                  __init__.py
__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.

In [ ]:
# configurations and main hyperparammeters
GROUP_BATCH_SIZE = 4000
SEED = 321
SPLITS = 5

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from multiprocessing import Pool

import numpy as np


class NearestNeighborsFeats(BaseEstimator, ClassifierMixin):
    '''
        This class should implement KNN features extraction 
    '''
    def __init__(self,  k_list, metric=None, n_jobs=1, n_classes=None, n_neighbors=None, eps=1e-6):
        self.n_jobs = n_jobs
        self.k_list = k_list
        self.metric = metric
        self.batch_size = 100_000
        if n_neighbors is None:
            self.n_neighbors = max(k_list) 
        else:
            self.n_neighbors = n_neighbors
            
        self.eps = eps        
        self.n_classes_ = n_classes
    
    def fit(self, X, y):
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list), metric=self.metric)
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list))
        self.NN.fit(X)        
        self.y_train = y
        self.n_classes = np.unique(y).shape[0] if self.n_classes_ is None else self.n_classes_
        
    def get_batches(self, batch_size, neighs_dist, neighs):
        n = len(neighs_dist)
        it = int(np.ceil(n/batch_size))

        batches = [(neighs_dist[batch_size*i:batch_size*(i+1)],
                    neighs[batch_size*i:batch_size*(i+1)]) for i in range(it)]
        return batches
        
    def predict(self, X):       
        neighs_dist, neighs = NN.kneighbors(X)
        batches = self.get_batches(self.batch_size, neighs_dist, neighs)
        with Pool(self.n_jobs) as p:
            test_feats = p.map(self.get_features_for_batch, batches)
                        
        return np.vstack(test_feats)
        
        
    def get_features_for_batch(self, neighs_dist, neighs):
        neighs_y = self.y_train[neighs] 

        return_list = []
        
        '''
            1. Labels distribution
        '''
        
        def bincount(x):
            return np.bincount(x, minlength=self.n_classes)
        
        for k in self.k_list:
            feats = np.apply_along_axis(bincount, axis=1, arr=neighs_y[:,:k])
            
            feats = feats/ feats.sum(axis=1)
            
            assert feats.shape == (neighs_dist.shape[0], self.n_classes)
            return_list += [feats]
            
        '''
            2. Mean
        '''
        
        for k in self.k_list:
            feats = [np.mean(neighs_y[:,:k], axis=1)]
            
            return_list += [feats]
        '''
            3. Quantilles
        '''
        
        q = np.arange(0.1, 1.0, 0.1)
        for k in self.k_list:
            feats = np.quantille(neighs_y[:,:k], q=q, axis=1).T
            
            assert feats.shape == (neighs_dist.shape[0], 9)
            return_list += [feats]
        
        knn_feats = np.hstack(return_list)
        return knn_feats

In [ ]:
X = np.random.randn(5_000_000, 10)
y = np.random.randint(10, size=(5_000_000))
nn_feats = NearestNeighborsFeats(k_list=[50, 100], n_jobs=8)
nn_feats.fit(X, y)
feats = nn_feats.predict(X)



In [ ]:
# read data
def read_data():
    train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test  = pd.read_csv('/kaggle/input/data-without-drift/test_clean.csv', dtype={'time': np.float32, 'signal': np.float32})
    sub  = pd.read_csv('/kaggle/input/liverpool-ion-switching/sample_submission.csv', dtype={'time': np.float32})

    return train, test, sub

# create batches of 4000 observations
def batching(df, batch_size):
    df['group'] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df

# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test

# get lead and lags features
def lag_with_pct_change(df, windows):
    for window in windows:    
        df['signal_shift_pos_' + str(window)] = df.groupby('group')['signal'].shift(window).fillna(0)
        df['signal_shift_neg_' + str(window)] = df.groupby('group')['signal'].shift(-1 * window).fillna(0)
    return df

# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size = batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2, 3])
    # create signal ** 2 (this is the new feature)
    df['signal_2'] = df['signal'] ** 2
    return df

# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'group', 'open_channels', 'time']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features


def run_cv_model_by_batch(train, test, splits, batch_col, feats, sample_submission, nn_epochs, nn_batch_size):
    
    seed_everything(SEED)
    
    oof_ = np.zeros((len(train), 11)) # build out of folds matrix with 11 columns, they represent our target variables classes (from 0 to 10)
    preds_ = np.zeros((len(test), 11))
    target = ['open_channels']
    group = train['group']
    kf = GroupKFold(n_splits=5)
    splits = [x for x in kf.split(train, train[target], group)]

    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[0])    
        new_split.append(sp[1])    
        new_splits.append(new_split)
    # pivot target columns to transform the net to a multiclass classification estructure (you can also leave it in 1 vector with sparsecategoricalcrossentropy loss function)
    tr = pd.concat([pd.get_dummies(train.open_channels), train[['group']]], axis=1)

    tr.columns = ['target_'+str(i) for i in range(11)] + ['group']
    target_cols = ['target_'+str(i) for i in range(11)]
    train_tr = np.array(list(tr.groupby('group').apply(lambda x: x[target_cols].values))).astype(np.float32)
    train = np.array(list(train.groupby('group').apply(lambda x: x[feats].values)))
    test = np.array(list(test.groupby('group').apply(lambda x: x[feats].values)))

    for n_fold, (tr_idx, val_idx, tr_orig_idx, val_orig_idx) in enumerate(new_splits[0:], start=0):
        
        tr_idx = tr_orig_idx
        val_idx = val_orig_idx
        
        train_x, train_y = train[tr_idx], train_tr[tr_idx]
        valid_x, valid_y = train[val_idx], train_tr[val_idx]
        print(f'Our training dataset shape is {train_x.shape}')
        print(f'Our validation dataset shape is {valid_x.shape}')

        gc.collect()
        
        
        
        preds_f = model.predict(valid_x)
        f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),  np.argmax(preds_f, axis=2).reshape(-1), average = 'macro') # need to get the class with the biggest probability
        
        print(f'Training fold {n_fold + 1} completed. macro f1 score : {f1_score_ :1.5f}')
        preds_f = preds_f.reshape(-1, preds_f.shape[-1])
        oof_[val_orig_idx,:] += preds_f
        te_preds = model.predict(test)
        te_preds = te_preds.reshape(-1, te_preds.shape[-1])           
        preds_ += te_preds / SPLITS
    # calculate the oof macro f1_score
    f1_score_ = f1_score(np.argmax(train_tr, axis = 2).reshape(-1),  np.argmax(oof_, axis = 1), average = 'macro') # axis 2 for the 3 Dimension array and axis 1 for the 2 Domension Array (extracting the best class)
    print(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')
    sample_submission['open_channels'] = np.argmax(preds_, axis = 1).astype(int)
    sample_submission.to_csv('submission_wavenet.csv', index=False, float_format='%.4f')
    
# this function run our entire program
def run_everything():
    
    print('Reading Data Started...')
    train, test, sample_submission = read_data()
    train, test = normalize(train, test)
    print('Reading and Normalizing Data Completed')
        
    print('Creating Features')
    print('Feature Engineering Started...')
    train = run_feat_engineering(train, batch_size = GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size = GROUP_BATCH_SIZE)
    train, test, features = feature_selection(train, test)
    print('Feature Engineering Completed...')
        
   
    print(f'Training Wavenet model with {SPLITS} folds of GroupKFold Started...')
    run_cv_model_by_batch(train, test, SPLITS, 'group', features, sample_submission, EPOCHS, NNBATCHSIZE)
    print('Training completed...')
        
# run_everything()

In [ ]:
%%time

KNN = 100
batch = 1000

test_pred = np.zeros((test.shape[0]),dtype=np.int8)
for g in [0,1,2,3,4]:
    print('Infering group %i'%g)
    
    # TRAIN DATA
    data = train.loc[train.group==g]
    X_train = np.zeros((len(data)-6,7))
    X_train[:,0] = 0.25*data.signal[:-6]
    X_train[:,1] = 0.5*data.signal[1:-5]
    X_train[:,2] = 1.0*data.signal[2:-4]
    X_train[:,3] = 4.0*data.signal[3:-3]
    X_train[:,4] = 1.0*data.signal[4:-2]
    X_train[:,5] = 0.5*data.signal[5:-1]
    X_train[:,6] = 0.25*data.signal[6:]
    y_train = data.open_channels[3:].values

    # TEST DATA
    data = test.loc[test.group==g]
    X_test = np.zeros((len(data)-6,7))
    X_test[:,0] = 0.25*data.signal[:-6]
    X_test[:,1] = 0.5*data.signal[1:-5]
    X_test[:,2] = 1.0*data.signal[2:-4]
    X_test[:,3] = 4.0*data.signal[3:-3]
    X_test[:,4] = 1.0*data.signal[4:-2]
    X_test[:,5] = 0.5*data.signal[5:-1]
    X_test[:,6] = 0.25*data.signal[6:]

    # HERE IS THE CORRECT WAY TO USE CUML KNN 
    #model = KNeighborsClassifier(n_neighbors=KNN)
    #model.fit(X_train,y_train)
    #y_hat = model.predict(X_test)
    #test_pred[test.group==g][1:-1] = y_hat
    #continue
    
    # WE DO THIS BECAUSE CUML v0.12.0 HAS A BUG
    model = NearestNeighbors(n_neighbors=KNN)
    model.fit(X_train)
    distances, indices = model.kneighbors(X_test)

    # FIND PREDICTIONS OURSELVES WITH STATS.MODE
    ct = indices.shape[0]
    pred = np.zeros((ct+6),dtype=np.int8)
    it = ct//batch + int(ct%batch!=0)
    #print('Processing %i batches:'%(it))
    for k in range(it):
        a = batch*k; b = batch*(k+1); b = min(ct,b)
        pred[a+3:b+3] = np.median( y_train[ indices[a:b].astype(int) ], axis=1)
        #print(k,', ',end='')
    #print()
    test_pred[test.group==g] = pred